# Optical Flow

Optical flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movement casued by the movement of the object or camera.

It has a few assumptions:
* The pixel intensitites of an object do not change between consecutive frames.
* Neighbouring pixels have similar motion.


The optical flow methods in OpenCV will first taken in a givens set of points from the user. The method will then attempt to find those points in the next frame.

OpenCV uses the Lucas-Kanade function. It computes optical flow for a sparse feature set (Only tracks the points that it was told to track). If we want to track all the points, wecan use the Gunner Farneback's algorithm to calculate dense optical flow. It colors the image black if no flow is detected.

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

Lucas-Kanade Method

In [3]:
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

In [4]:
lk_params = dict(winSize = (200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [5]:
cap = cv2.VideoCapture(0)
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

#Points to track
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)
#Creates a mask of size of the previous frame
mask = np.zeros_like(prev_frame)

while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_prev), int(y_prev)), (0,255,0), 3)

        frame = cv2.circle(frame, (int(x_new), int(y_new)), 8, (0,0,255), -1)

    img = cv2.add(frame, mask)
    cv2.imshow('tracking', img)

    k = cv2.waitKey(30)
    if k == 27:
        break

    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
cap.release()

Dense Optical Flow

In [6]:
cap = cv2.VideoCapture(0)

ret, frame1 = cap.read()

prevImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)

hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prevImg, nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    #Convert to polar coordinates
    mag, ang = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees=True)

    hsv_mask[:, :, 0] = ang/2

    hsv_mask[:, :, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

    bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)

    cv2.imshow('frame', bgr)

    k = cv2.waitKey(10) & 0xFF

    if k == 27:
        break

    prevImg = nextImg

cv2.destroyAllWindows()
cap.release()


# MeanShift and CamShift Tracking

Gets all points into clusters and get the points to cluster towards their respective centers every iteration until all clusters have converged and there is no more movement.

MeanShift can be given a target to track, calculate the color histogram of the target area, and then keep sliding the tracking window to the closest match (Cluster center).

In [12]:
cap = cv2.VideoCapture(0)

ret, frame = cap.read()

face_cascade = cv2.CascadeClassifier('Images/haarcascade_frontalface_default.xml')

face_rects = face_cascade.detectMultiScale(frame)

(face_x, face_y, w, h) = tuple(face_rects[0])

track_window = (face_x, face_y, w, h)

roi = frame[face_y:face_y+h, face_x:face_x+w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])

cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()

    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180], 1)

        #ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, [pts], True, (0,0,255), 3)


        #x, y, w, h = track_window
        #img2 = cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 0, 255), 5)

        cv2.imshow('img', img2)

        k = cv2.waitKey(1) & 0xFF 
        if k == 27:
            break
    else:
        break      

cv2.destroyAllWindows()
cap.release()